# Processing of data collection

## Getting songs IDs

Getting the song name and id from a playlist with 100 songs that I like. And  75 songs I don't like.<br>
First I generated playlist.py(ids folder) using [Get Playlist's items API](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-playlists-tracks) to get the songs' ids through the shell and pass it to a python file in order to get the individual songs id.<br>

## Generating Songs IDs
Then using [Spotify's Audio Feature for several tracks API](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-several-audio-features)
I generated a JSON(data folder) with the shell to get the audio features from the tracks<br>
From [Spotify documentation](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features):
- Acousticness
    - A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
- danceability
    - Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
- duration_ms
    - The duration of the track in milliseconds.
- energy
    - Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
- instrumentalness
     - Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
- liveness
    - Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.
- loudness
    - The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.
- mode
    - Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
- speechiness
    - Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
- tempo
    - The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
- time_signature
    - An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure).

# Library Import

In [2]:
import pandas as pd
import numpy as np
import cufflinks as cf
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import json

# Data Cleaning

In [3]:
#return a dict of the json
with open('./data/liked_ids.json', 'r') as f:
    likes = json.load(f)

with open('./data/disliked_ids.json', 'r') as f:
    dislikes = json.load(f)

In [4]:
likes = pd.DataFrame(likes['audio_features'])
likes = likes.drop(["type", "id", "uri", "track_href", "analysis_url", "time_signature"], axis=1)
likes

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.643,0.594,9,-5.191,0,0.0312,0.06950,0.000015,0.1150,0.195,118.037,250142
1,0.440,0.465,3,-6.592,1,0.0318,0.68900,0.000001,0.3840,0.241,126.937,310394
2,0.546,0.640,8,-5.582,1,0.3700,0.04380,0.000000,0.1370,0.630,137.728,339250
3,0.711,0.611,1,-5.453,1,0.3290,0.00575,0.000000,0.2310,0.144,134.140,252070
4,0.746,0.798,0,-8.382,1,0.3490,0.13700,0.000000,0.1390,0.553,109.968,214827
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.812,0.714,11,-8.045,0,0.3060,0.00806,0.000000,0.1790,0.690,94.612,235200
96,0.792,0.588,1,-9.319,0,0.0770,0.09280,0.001550,0.0656,0.731,119.950,206253
97,0.860,0.791,0,-4.784,1,0.0568,0.16900,0.000106,0.1060,0.887,92.023,175918
98,0.767,0.379,0,-10.348,1,0.0385,0.66800,0.000145,0.2170,0.252,103.956,230578


In [5]:
likes["is_likeable"] = 1

In [6]:
dislikes = pd.DataFrame(dislikes['audio_features'])
dislikes = dislikes.drop(["type", "id", "uri", "track_href", "analysis_url", "time_signature"], axis=1)
dislikes

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.551,0.736,9,-3.731,0,0.2000,0.074800,0.000426,0.2900,0.481,142.286,178500
1,0.389,0.519,6,-6.400,0,0.0340,0.478000,0.000000,0.2130,0.723,181.728,310581
2,0.837,0.611,7,-4.127,1,0.0761,0.347000,0.000040,0.0497,0.429,99.974,232080
3,0.761,0.359,11,-11.559,0,0.0730,0.851000,0.510000,0.1130,0.384,118.000,431049
4,0.897,0.597,10,-4.032,0,0.0575,0.052100,0.000000,0.1140,0.403,113.005,186828
...,...,...,...,...,...,...,...,...,...,...,...,...
70,0.389,0.665,0,-6.169,1,0.0644,0.178000,0.000732,0.1160,0.199,117.055,193547
71,0.526,0.882,9,-4.006,1,0.0509,0.001400,0.000000,0.1440,0.424,144.982,211493
72,0.388,0.633,9,-4.769,1,0.0275,0.033900,0.000000,0.1040,0.381,173.947,252440
73,0.483,0.961,4,-3.314,1,0.0683,0.000842,0.000015,0.3470,0.177,98.526,183107


In [7]:
dislikes["is_likeable"] = 0 

In [8]:
df = pd.concat([likes, dislikes])

In [9]:
df = shuffle(df)

In [10]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,is_likeable
8,0.740,0.682,9,-10.420,0,0.0504,0.06920,0.530000,0.3810,0.410,113.201,544627,1
87,0.725,0.838,6,-7.486,0,0.0312,0.01270,0.312000,0.1010,0.414,140.053,404667,1
74,0.685,0.848,8,-4.561,0,0.0858,0.08610,0.000007,0.0783,0.761,179.870,163084,1
50,0.760,0.477,4,-7.645,1,0.1110,0.86000,0.000426,0.1010,0.460,118.107,187039,1
95,0.812,0.714,11,-8.045,0,0.3060,0.00806,0.000000,0.1790,0.690,94.612,235200,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,0.794,0.669,6,-5.837,0,0.1440,0.06300,0.000062,0.0997,0.128,114.960,196587,1
65,0.592,0.403,0,-8.768,0,0.4140,0.62000,0.000000,0.5400,0.391,105.169,320680,1
32,0.725,0.860,1,-3.726,1,0.1820,0.42200,0.000284,0.0674,0.791,175.896,230125,1
70,0.389,0.665,0,-6.169,1,0.0644,0.17800,0.000732,0.1160,0.199,117.055,193547,0


In [15]:
df.dtypes

danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms           int64
is_likeable           int64
dtype: object

In [26]:
target = 'is_likeable'
disc = ["mode"]
cont = ['danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms']


,mode
8,0
87,0
74,0
50,1
95,0
...,...
31,0
65,0
32,1
70,1


In [13]:
df["mode"].unique()

array([0, 1])